In [1]:
import pickle
import pandas as pd
import time
import json
import requests
import re
from lxml import html
from bs4 import BeautifulSoup
import csv
import tqdm
from urllib.parse import urljoin
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

options = Options()
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('start-maximized')
options.add_argument('window-size=1920x1080')  # Indstil en fast vinduesstørrelse

In [2]:
article_links = [
    "https://www.berlingske.dk/aktier/fredagens-aktier-vestas-kravlede-op-i-rekordhoejt-c25-indeks",
    "https://www.berlingske.dk/aktier/fredagens-aktier-vestas-kravlede-op-i-rekordhoejt-c25-indeks",
    "https://www.berlingske.dk/virksomheder/vestas-har-aldrig-haft-en-stoerre-forretning-alligevel-er-aktiekursen",
    "https://www.berlingske.dk/virksomheder/afsloering-chefredaktoer-og-hans-medie-skriver-positivt-om",
    "https://www.berlingske.dk/aktier/aktier-genmab-vil-igen-tage-fokus-efter-stor-kurslussing",
    "https://www.berlingske.dk/aktier/aktier-maersk-og-c25-gaar-lidt-i-roedt-efter-handelskrigens-start",
    "https://www.berlingske.dk/virksomheder/vestas-og-mitsubishi-kritiseres-for-valg-af-nye-offshore-topchefer",
    "https://www.berlingske.dk/aktier/boersbarometer-vi-ses-om-ni-maaneder-til-den-tid-kan-alt-vaere-forandret",
    "https://www.berlingske.dk/aktier/onsdagens-aktier-gn-og-ambu-kaemper-om-topplacering-i-groent-marked",
    "https://www.berlingske.dk/aktier/aktier-negativ-aabning-novo-dropper",
    "https://www.berlingske.dk/virksomheder/fyret-ambu-topchef-faar-mindst-300-mio.-kr.-med-ud-ad-doeren",
    "https://www.berlingske.dk/aktier/kurstaesk-til-c25-skelsaettende-rentemoede-lammer-aktierne",
    "https://www.berlingske.dk/aktier/lille-dansk-investor-slaar-storbanker",
    "https://www.berlingske.dk/aktier/lundbeck-direktoer-saelger-aktier-for-naesten-9-mio.-kr",
    "https://www.berlingske.dk/aktier/torsdagens-aktier-unibrew-i-top-og-pandora-i-bund-i-svagt-negativt-c25",
    "https://www.berlingske.dk/oekonomi/torsdagens-aktier-opjustering-sikrede-coloplast-stort-kurshop",
    "https://www.berlingske.dk/aktier/aktier-chr.-hansen-falder-tungt-i-negativ-aabning",
    "https://www.berlingske.dk/aktier/mandagens-aktier-ambu-toppede-c25-efter-gode-nyheder-fra-usa",
    "https://www.berlingske.dk/virksomheder/topdanmark-lavere-investeringer-rammer-bundlinjen",
    "https://www.berlingske.dk/virksomheder/danske-giganter-brager-igennem-krisen-det-er-virkelig",
    "https://www.berlingske.dk/aktier/renten-dansede-paa-stedet-i-stille-marked",
    "https://www.radio24syv.dk/aktier/millionaerklubben-12-01-2018",
    "https://www.berlingske.dk/virksomheder/kaempekrise-og-slagsmaal-om-milliardregning-stopper-metrobyggeri",
    "http://www.business.dk/aktier/live",
    "https://www.berlingske.dk/business/topadvokaten-bygger-skoler-i-nepal",
    "https://www.berlingske.dk/virksomheder/velux-i-digital-satsning-med-apple-vil-have-huse-til-at-taenke-selv",
    "https://www.berlingske.dk/business/se-billederne-dansk-tobakskaempe-gaar-paa-boersen",
    "https://www.berlingske.dk/business/saa-mange-bestyrelsesposter-har-c20-topcheferne",
    "https://www.radio24syv.dk/aktier/millionaerklubben-12-01-2018",
    "https://www.berlingske.dk/virksomheder/saadan-bor-de-danske-c20-selskaber",
    "http://schroeder.blogs.business.dk/aktier/",
    "http://aktiemanager.business.dk/aktier/nsm-fall-2015",
    "https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception",
    "https://www.berlingske.dk/virksomheder/virtuel-valuta-har-gjort-niklas-til-mangemillionaer",
    "https://www.berlingske.dk/business/saa-mange-c20-topchefer-var-danskere-i-2004-og-saa-faa-er-der-nu",
    "https://www.berlingske.dk/virksomheder/yousee-direktoer-jeg-vaagner-med-ondt-i-maven-hver-morgen",
    "https://www.berlingske.dk/business/saadan-klarer-c20-selskaberne-sig-i-pixi-udgave",
    "http://kimvalentin.blogs.business.dk/business/",
    "https://www.berlingske.dk/business/c20-chefernes-karakterbog",
    "https://www.berlingske.dk/virksomheder/mercedes-skyder-kaempe-millionbeloeb-i-skype-milliardaers-robotter",
    "http://www.business.dk/business/manden-vi-kaldte-kasi-jesper",
    "https://www.berlingske.dk/business/saa-meget-tjener-de-danske-topchefer",
    "https://www.berlingske.dk/virksomheder/vi-goer-det-modsatte-af-andre-for-saa-fremstaar-man-federe",
    "http://www.business.dk/business/live",
    "https://www.business.dk/business/live",
    "https://www.berlingske.dk/business/her-er-de-danske-aktier-som-har-faaet-prygl-i-2016",
    "http://www.business.dk/aktier/aktieboble-c20-aktier-fortsaetter-i-vejret",
    "https://www.berlingske.dk/virksomheder/brandingekspert-nykredit-er-den-nye-gebyrgrib",
    "http://www.radio24syv.dk/aktier/tips-til-investering"
]

# To output the list
article_links

['https://www.berlingske.dk/aktier/fredagens-aktier-vestas-kravlede-op-i-rekordhoejt-c25-indeks',
 'https://www.berlingske.dk/aktier/fredagens-aktier-vestas-kravlede-op-i-rekordhoejt-c25-indeks',
 'https://www.berlingske.dk/virksomheder/vestas-har-aldrig-haft-en-stoerre-forretning-alligevel-er-aktiekursen',
 'https://www.berlingske.dk/virksomheder/afsloering-chefredaktoer-og-hans-medie-skriver-positivt-om',
 'https://www.berlingske.dk/aktier/aktier-genmab-vil-igen-tage-fokus-efter-stor-kurslussing',
 'https://www.berlingske.dk/aktier/aktier-maersk-og-c25-gaar-lidt-i-roedt-efter-handelskrigens-start',
 'https://www.berlingske.dk/virksomheder/vestas-og-mitsubishi-kritiseres-for-valg-af-nye-offshore-topchefer',
 'https://www.berlingske.dk/aktier/boersbarometer-vi-ses-om-ni-maaneder-til-den-tid-kan-alt-vaere-forandret',
 'https://www.berlingske.dk/aktier/onsdagens-aktier-gn-og-ambu-kaemper-om-topplacering-i-groent-marked',
 'https://www.berlingske.dk/aktier/aktier-negativ-aabning-novo-drop

In [3]:
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.implicitly_wait(15)  # Vent op til 10 sekunder på, at elementer bliver tilgængelige

# Dine login oplysninger
login = "jvgronlund@yahoo.com"
password = "rebklemmer72"

# Gå til Berlingskes login-side
driver.get('https://www.berlingske.dk/login')

# Afvis cookie-popup (hvis den er der)
try:
    accept_cookies_button = driver.find_element(By.CSS_SELECTOR, 'button[id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')
    accept_cookies_button.click()
    time.sleep(2)
except Exception as e:
    print("Ingen cookie-popup fundet, fortsætter uden.")

# Find login-felterne og indtast login-oplysninger
email_field = driver.find_element(By.ID, 'email')
password_field = driver.find_element(By.ID, 'current-password')

email_field.send_keys(login)
password_field.send_keys(password)

# Find og klik på login-knappen
login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
login_button.click()

# Vent lidt for at sikre login-succes
time.sleep(5)

In [4]:
# Tom liste til at gemme artikeldetaljer
articles = []

# Loop gennem alle artikel-links
for link in tqdm.tqdm(article_links):
    try:
        # Brug Selenium driver til at navigere til siden
        driver.get(link)

        # Vent indtil siden er færdigindlæst og overskriften er tilgængelig
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))
        
        # Scroll ned ad siden for at sikre, at alt indhold bliver indlæst
        time.sleep(2)  # Giv siden lidt tid til at indlæse
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Vent lidt for at sikre, at alt indlæses
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Hent overskriften (h2 er ofte brugt til hovedtitlen)
        title_element = driver.find_element(By.TAG_NAME, 'h2')
        title = title_element.text if title_element else 'No Title'
        
        # Hent datoen (tilpasset til den faktiske HTML-struktur på siden)
        date_element = driver.find_element(By.CLASS_NAME, 'article-byline__date')
        date = date_element.text if date_element else 'No Date'

        # Prøv først at hente hele artiklen ved at kombinere elementer fra forskellige sektioner
        try:
            intro_paragraph = driver.find_element(By.CSS_SELECTOR, 'p.article-header__intro').text
        except:
            intro_paragraph = ""

        # Prøv at hente teksten fra "articleBody"
        try:
            article_body = driver.find_element(By.ID, "articleBody")
            inner_html = article_body.get_attribute('innerHTML')
            soup = BeautifulSoup(inner_html, 'html.parser')
            paragraphs = soup.find_all('p')
            article_text = "\n".join([paragraph.get_text() for paragraph in paragraphs])
        except:
            article_text = ""

        # Kombinér intro-tekst og artikel-tekst fra "articleBody"
        full_article_text = f"{intro_paragraph}\n{article_text}" if intro_paragraph else article_text

        # Hvis teksten stadig er mangelfuld, prøv at finde alternative sektioner (som i HTML1)
        if not full_article_text or len(full_article_text) < 100:  # Brug en passende tærskel
            alternative_sections = driver.find_elements(By.CSS_SELECTOR, 'div.article__section p')
            if alternative_sections:
                alternative_text = "\n".join([section.text for section in alternative_sections])
                if alternative_text:
                    full_article_text = f"{intro_paragraph}\n{alternative_text}" if intro_paragraph else alternative_text
        
        # Tilføj detaljerne til listen
        articles.append({
            'URL': link,
            'Title': title,
            'Date': date,
            'Content': full_article_text
        })

    except TimeoutException:
        print(f"Timeout while trying to load the page: {link}")
    except Exception as e:
        print(f"An error occurred for {link}: {e}")

# Luk driveren, når alle sider er blevet besøgt
driver.quit()

# Konverter listen af artikeldetaljer til en pandas DataFrame
df = pd.DataFrame(articles)

 45%|████████████████████████████████████▊                                             | 22/49 [03:33<07:06, 15.80s/it]

Timeout while trying to load the page: https://www.radio24syv.dk/aktier/millionaerklubben-12-01-2018


 47%|██████████████████████████████████████▍                                           | 23/49 [04:07<09:08, 21.11s/it]

Timeout while trying to load the page: https://www.berlingske.dk/virksomheder/kaempekrise-og-slagsmaal-om-milliardregning-stopper-metrobyggeri


 49%|████████████████████████████████████████▏                                         | 24/49 [04:38<10:05, 24.20s/it]

...

Timeout while trying to load the page: http://www.business.dk/aktier/aktieboble-c20-aktier-fortsaetter-i-vejret


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 48/49 [16:14<00:31, 31.78s/it]

Timeout while trying to load the page: https://www.berlingske.dk/virksomheder/brandingekspert-nykredit-er-den-nye-gebyrgrib


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [16:45<00:00, 20.52s/it]

Timeout while trying to load the page: http://www.radio24syv.dk/aktier/tips-til-investering


In [5]:
# Juster Pandas-indstillinger for at vise hele indholdet i 'Content' kolonnen
pd.set_option('display.max_colwidth', None)

# Vis DataFrame
df

URL  \
0                  https://www.berlingske.dk/aktier/fredagens-aktier-vestas-kravlede-op-i-rekordhoejt-c25-indeks   
1                  https://www.berlingske.dk/aktier/fredagens-aktier-vestas-kravlede-op-i-rekordhoejt-c25-indeks   
2   https://www.berlingske.dk/virksomheder/vestas-har-aldrig-haft-en-stoerre-forretning-alligevel-er-aktiekursen   
3              https://www.berlingske.dk/virksomheder/afsloering-chefredaktoer-og-hans-medie-skriver-positivt-om   
4                      https://www.berlingske.dk/aktier/aktier-genmab-vil-igen-tage-fokus-efter-stor-kurslussing   
5             https://www.berlingske.dk/aktier/aktier-maersk-og-c25-gaar-lidt-i-roedt-efter-handelskrigens-start   
6      https://www.berlingske.dk/virksomheder/vestas-og-mitsubishi-kritiseres-for-valg-af-nye-offshore-topchefer   
7      https://www.berlingske.dk/aktier/boersbarometer-vi-ses-om-ni-maaneder-til-den-tid-kan-alt-vaere-forandret   
8           https://www.berlingske.dk/aktier/onsdagens-aktier-gn-og-ambu-kaemper-om-topplacering-i-groent-marked   
9                                           https://www.berlingske.dk/aktier/aktier-negativ-aabning-novo-dropper   
10           https://www.berlingske.dk/virksomheder/fyret-ambu-topchef-faar-mindst-300-mio.-kr.-med-ud-ad-doeren   
11                   https://www.berlingske.dk/aktier/kurstaesk-til-c25-skelsaettende-rentemoede-lammer-aktierne   
12                                        https://www.berlingske.dk/aktier/lille-dansk-investor-slaar-storbanker   
13                      https://www.berlingske.dk/aktier/lundbeck-direktoer-saelger-aktier-for-naesten-9-mio.-kr   
14       https://www.berlingske.dk/aktier/torsdagens-aktier-unibrew-i-top-og-pandora-i-bund-i-svagt-negativt-c25   
15              https://www.berlingske.dk/oekonomi/torsdagens-aktier-opjustering-sikrede-coloplast-stort-kurshop   
16                            https://www.berlingske.dk/aktier/aktier-chr.-hansen-falder-tungt-i-negativ-aabning   
17                 https://www.berlingske.dk/aktier/mandagens-aktier-ambu-toppede-c25-efter-gode-nyheder-fra-usa   
18                      https://www.berlingske.dk/virksomheder/topdanmark-lavere-investeringer-rammer-bundlinjen   
19                  https://www.berlingske.dk/virksomheder/danske-giganter-brager-igennem-krisen-det-er-virkelig   
20                                    https://www.berlingske.dk/aktier/renten-dansede-paa-stedet-i-stille-marked   

                                                                                                Title  \
0                                        Fredagens aktier: Vestas kravlede op i rekordhøjt C25-indeks   
1                                        Fredagens aktier: Vestas kravlede op i rekordhøjt C25-indeks   
2              Vestas har aldrig haft en større forretning – alligevel er aktiekursen savet midt over   
3   Afsløring: Chefredaktør og hans medie skriver positivt om virksomheder – og har selv aktier i dem   
4                                           Aktier: Genmab vil igen tage fokus efter stor kurslussing   
5                                     Aktier: Mærsk og C25 går lidt i rødt efter handelskrigens start   
6                                  Vestas og Mitsubishi kritiseres for valg af nye offshore-topchefer   
7            Børsbarometer: Vi ses om ni måneder. Til den tid kan alt være forandret på aktiemarkedet   
8                                  Onsdagens aktier: GN og Ambu kæmper om topplacering i grønt marked   
9                                                               Aktier: Negativ åbning - Novo dropper   
10                                         Fyret Ambu-topchef får mindst 300 mio. kr. med ud ad døren   
11                                         Kurstæsk til C25: »Skelsættende« rentemøde lammer aktierne   
12                                                               Lille dansk investor slår storbanker   
13                                              Lundbeck-direktør sælger aktier for næ

In [6]:
# Gem DataFrame til en CSV-fil, hvis ønsket
df.to_csv('berlingske_RestLinks_Endelig.csv', index=False)